Investigate the distribution of the test statistic

$$ 
\widehat{T}_{N,k} =  \exp \left\{\widehat{H}_{N, k, \textit{q}}\right \} \big{/} 
\sqrt{\det {\frac{\nu -2}{\nu}}\widehat{C}_N } - c_0(\nu, m)
$$
converge to zero in probability as $N \to \infty$ for any fixed $k \geq 1$, where $\widehat{H}_{N, k, \textit{q}}$


$$
\widehat{H}_{N,k,\textit{q}} = \frac{1}{1-\textit{q}}\log 
\left[\frac{1}{N}\sum_{i=1}^{N}\left(\zeta_{N,i,k}\right)^{1-\textit{q}} \right]= 
\frac{1}{1-\textit{q}}\log \left[\widehat{I}_{N,i, {k}} \right]
$$

and $\widehat{C}_N$ is defined as

$$
  \widehat{C}_N = \left( \frac{1}{N-1} \sum_{r=1}^{N} X_r^{(i)}X_r^{(j)}    
 \right)_{1 \leq {i}, {j} \leq {N}} = \frac{1}{N-1} \sum_{r=1}^{N}X_r X_r^{\prime},
$$


and also $c_0(\nu, m)$  is defined 

$$
 c_0(\nu, m) = \frac{B^{\frac{1}{1-\textit{q}}} \left(\frac{\textit{q}(m + \nu)}{2} - \frac{m}{2}, 
\frac{m}{2}\right)(\pi \nu)^{\frac{m}{2}}}{B^{\frac{\textit{q}}{1-\textit{q}}}(\frac{\nu}{2}, 
\frac{m}{2})\Gamma \left(\frac{m}{2} \right)},
$$

for various values of the parameters
    $N$ - number of points (sample size)
    $m$ - dimension
    $k$ - nearest neighbour index
    $\nu$ - degrees of freedom

N = 1000, 2000, ..., 100000
m = 1,2,3
k = 1,2,3
$\nu$ = 3, 4, 5,

# Preliminaries

In [345]:
# import packages
import numpy as np
import scipy as sc
import scipy.stats as st
import matplotlib.pyplot as plt
import matplotlib.ticker as tkr

# Golden ratio (for nice plotting)
GR = (1+np.sqrt(5))/2

In [346]:
def create_points(npts, dist = "T", dim=2, dof=3, scale=1, cov_mtx = None, standardize=False):
    '''
    Generate points from a specified distribution.
    Inputs
        npts: number of points
        dist: the distribution of the points
            G:  Gaussian
            T:  Stud...
            L : Multivariate Laplace 
            
    '''
    if cov_mtx:
        dim = len(cov_mtx)
    else:
        cov_mtx = np.identity(dim)
    
    # init
    points = None

    if dist == "G":
        points = scale*np.random.multivariate_normal([0]*dim, cov_mtx, npts)


    elif dist == "T":

        # X = d = Z/sqrt(V/nu) where Z~N(0,Sigma) and V~chisquared(nu)
        zpts = np.random.multivariate_normal([0]*dim, cov_mtx, npts)
        csq = np.tile(np.random.gamma(dof/2.0, 2.0/dof, npts),(dim,1)).T
        points = zpts/np.sqrt(csq)
        sfactor = dof/(dof-2) # variance
        points = scale*points/np.sqrt(sfactor)
        
        
        
    elif dist == "L":
        
        #Z = d = sqrt(V_0).X, where X ~ N_m(0,C) and V_0 is exponential
        Xpts = np.random.multivariate_normal([0]*dim, cov_mtx, npts)
        V_vals = np.random.exponential(scale = 1, size=npts)
        Vpts = np.sqrt(V_vals )
        points = np.multiply(Xpts, Vpts[:, np.newaxis]) if dim > 1 else np.multiply(Xpts, Vpts)
        
        
        
        #mean = [0]*dim
        #x_m = npts - mean
        #exp = np.exp(-(np.linalg.solve(cov_mtx, x_m)).T.dot(x_m))
        #gamma = np.random.gamma((dim+1)/2)
        #detC =  np.linalg.det(cov_mtx)
        #points = (1. / (2** dim) * (np.pi ** ((dim-1)/2))*(gamma)*(detC))*(exp)

        


    else :
        print('Distribution not recognised')
        
    if standardize:
        points = (points - np.mean(points, axis=0))/np.std(points, axis=0)
    
    return points




In [347]:
# Test
pts1 = create_points(npts=11, dist='T', dim=2, dof=4)
print(pts1)

[[-0.24173068 -0.00953813]
 [-2.12830005  1.06854131]
 [ 0.70965655 -0.04945681]
 [-0.41818044  0.47634586]
 [ 0.43066845 -0.20047532]
 [ 1.50532823 -0.93384945]
 [-0.26238961 -1.05722882]
 [-0.97839841  0.20147585]
 [ 0.85517191  0.24823098]
 [-2.56606501 -1.08506665]
 [ 0.02559686  0.29259978]]


In [348]:
# Test
#pts2 = create_points(npts=111, dist='G', dim=2)
#print(pts2)

## Estimate the entropy of Renyi

$$\widehat{H}_{N,k,q} = \frac{1}{1-q}\log \left[\frac{1}{N}\sum_{i=1}^{N}%
\left(\zeta_{N,i,k}\right)^{1-q} \right] $$ where
$$\zeta_{N,i,k} = (N-1)C_{k} {V_m} (\rho_{k,N-1}^{(i)})^m,$$  
$$C_k = \left[\frac{\Gamma(k)}{\Gamma(k+1-q)}\right]^{\frac{1}{1-q}},$$ 
$$V_m = \pi^{\frac{m}{2}}/\Gamma(\frac{m}{2} + 1)$$ and 
$$\bar{\rho} = \left\{\prod_{i=1}^{N}\rho_{1,N-1}^{(i)}\right\}^{\frac{1}{N}}.$$
$$ q = 1- \frac{2}{dof-m}$$

In [349]:
def compute_entropy_estimate(points, k=1, dof = 23):

    # define harmonic numbers
    # def harmonic_number(n):
    # return sum([1/i for i in range(1,n+1)])

    
    # compute near neighbour distances
    from sklearn.neighbors import KDTree
    tree = KDTree(points)
    dist, ind = tree.query(points, k=k+1)
    nnd = dist[:,k]
    gmean_nnd = sc.stats.mstats.gmean(nnd)
    
    # extract dimensions 
    npts = points.shape[0]
    m = points.shape[1]
    
    # compute q
    q =  1 - 2 / (dof - m)

    # compute volume of unit ball 
    vub = (np.pi**(m/2))/(sc.special.gamma(m/2 + 1))
    
    # compute  the value of C_k 
    C_k = (sc.special.gamma(k) / sc.special.gamma(k + 1 - q)) ** (1 / (1-q))
    
    
    # compute  the value of zeta
    zeta = ((npts-1) *(C_k ) * (vub) * (gmean_nnd) ** m)**(1-q)
    
    # compute and return estimate
    est = (1 / (1-q) )* np.log((np.sum(zeta ))/npts)

    return(est)

In [350]:
# num_points = points.shape[0]
# m = points.shape[1]

In [351]:
C_k = (sc.special.gamma(k) / sc.special.gamma(k + 1 - q)) ** (1 / (1-q))
C_k

1.6525208379000702

In [352]:

vub = (np.pi**(m/2))/(sc.special.gamma(m/2 + 1))
vub

3.141592653589793

In [353]:
points = create_points(npts=10000)
demo = compute_entropy_estimate(points, k=1, dof=23)
demo

-94.36782626523501

In [335]:
#q = 0.6
#k = 1
#C_k = (sc.special.gamma(k) / sc.special.gamma(k + 1 - q)) ** (1 / (1-q))
#C_k    

$$
\sqrt{\det {\frac{\nu -2}{\nu}}\widehat{C}_N }
$$

$$
  \widehat{C}_N  = \frac{1}{N-1} \sum_{r=1}^{N}X_r X_r^{\prime},
$$

In [336]:
# compute C_N(hat)

#X_r = np.arange(npts)
#X_r =np.random.randn(npts)
dof=23
X_r =   create_points(npts =100, dist='G', dim=1)
X_m = X_r.transpose()
product = np.outer(X_r , X_m)

for i in range(npts):
    X_r =  create_points(npts=100, dist='G', dim=1)
    X_m = X_r.transpose()
    product = product + np.outer(X_r , X_m)
Rowsum = product
C_N = (Rowsum)/(npts - 1) 
C_N2 = ((dof-2)/dof)*C_N
C_N3 = np.linalg.det(C_N2)
const = np.sqrt(C_N3)
const
    

0.0012433135874700034

### Test Statistic

$$ 
\widehat{T}_{N,k} =  \exp \left\{\widehat{H}_{N, k, \textit{q}}\right \} \big{/} 
\sqrt{\det {\frac{\nu -2}{\nu}}\widehat{C}_N } - c_0(\nu, m)
$$
where $\widehat{C}_N$ is defined as

$$
  \widehat{C}_N  = \frac{1}{N-1} \sum_{r=1}^{N}X_r X_r^{\prime},
$$
and 
$$
 c_0(\nu, m) = \frac{B^{\frac{1}{1-\textit{q}}} \left(\frac{\textit{q}(m + \nu)}{2} - \frac{m}{2}, 
\frac{m}{2}\right)(\pi \nu)^{\frac{m}{2}}}{B^{\frac{\textit{q}}{1-\textit{q}}}(\frac{\nu}{2}, 
\frac{m}{2})\Gamma \left(\frac{m}{2} \right)}.
$$



In [337]:
# def compute_q(dof, m=2):
#    q = 1 - 2 / (dof - m)
#    return q     

In [338]:
# dof_v = [5,6,7,8,9,10,11,12,13]
# for dof in dof_v:
#    demo = compute_q(dof, m=2)
#    print(demo)

In [339]:
def compute_test_statistic(points, k=1):
    
    # extract parameters 
    npts, m = points.shape
    
    # estimate entropy
    entropy_estimate = compute_entropy_estimate(points, k, dof = 23)
    
    
                     
    # compute C_N(hat) and const
    X_r =   create_points(npts =100, dist='G', dim=1)
    X_m = X_r.transpose()
    product = np.outer(X_r , X_m)

    for i in range(npts):
        X_r =  create_points(npts=100, dist='G', dim=1)
        X_m = X_r.transpose()
        product = product + np.outer(X_r , X_m)
    Rowsum = product
    C_N = (Rowsum)/(npts - 1) 
    C_N2 = ((dof-2)/dof)*C_N
    C_N3 = np.linalg.det(C_N2)
    const = np.sqrt(C_N3)
                 
                     
    
    # compute quotient
    quot = (np.e**entropy_estimate)/(const) 
                     
                     
    # compute q
    q = 1 - 2 / (dof - m)
    

    # compute constant c_0(v,m)
    num = (sc.special.beta(q*(m + dof)/2 - m/2, m/2))**(1/(1-q))*(np.pi * dof) ** (m/2)
    deno = ((sc.special.beta(dof/2, m/2)) ** (q/(1-q)))*(sc.special.gamma(m/2))
    c_0 =  num / deno
                     
    
    # return test statistic
    return quot - c_0




In [340]:
entropy_estimate = compute_entropy_estimate(points, k)
quot = (np.e**entropy_estimate)/(const) 
quot

8.203599218271246e-39

In [341]:
demo = compute_test_statistic(points, k=1)
demo

-19.792553591665865

In [342]:
c_0

19.792553591665865

In [343]:
dof = 23
m= 2
q = 1 - 2 / (dof - m)
num = (sc.special.beta(q*(m + dof)/2 - m/2, m/2))**(1/(1-q))*(np.pi * dof) ** (m/2)
deno = (sc.special.beta(dof/2, m/2)) ** (q/(1-q))*(sc.special.gamma(m/2))
c_0 =  num / deno
c_0

19.792553591665865

In [344]:
# compute test statistic T(m,N,k,q) for T(8)  and Laplace(2)
# should be zero for GG(1.0)
#
npts = 1000
nrep = 100
#q = 0.6
results = np.zeros((nrep,2))
for rep in range(nrep):
    ptsT  = create_points(npts, dist='T', dim=2, dof=23)
    ptsL = create_points(npts, dist='L', dim=2)
    
    valT = compute_test_statistic(ptsT)    
    valL = compute_test_statistic(ptsL)
    results[rep,:] = [valT, valL]

mT = np.mean(results[:,0])
sT = np.std(results[:,0])
print('T:  {:.4f}, {:.4f}'.format(mT, sT))
mL = np.mean(results[:,1])
sL = np.std(results[:,1])
print('L: {:.4f}, {:.4f}'.format(mL, sL))

T:  -19.7926, 0.0000
L: -19.7926, 0.0000
